In [ ]:
import numpy as np
import csv
import pandas as pd
import datetime 
from pandas_profiling import ProfileReport

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')
path = '/content/drive/My Drive/Trabajo de título/TT2/Archivos de respaldo'

#Obtención de datos respecto a la variable objetivo "RESPONDIDA"

In [ ]:
from os import sep
from numpy.core.numeric import NaN
import pandas as pd
import datetime as dt
import numpy as np
from pandas.io.parsers import count_empty_vals

data = pd.read_csv(path + '/DatasetV4_2.csv', delimiter = ';')
data_respuesta = pd.read_csv(path + '/BD_Respuesta2020.csv', delimiter = ';')

data['Fecha'] =  pd.to_datetime(data['Fecha'])
data['Fecha_Termino'] =  pd.to_datetime(data['Fecha_Termino'])

data_respuesta['FECHAFIN'] =  pd.to_datetime(data_respuesta['FECHAFIN'])

data_respuesta['FECHAFIN'] = data_respuesta['FECHAFIN'].dt.strftime('%Y-%m-%d').astype(str)

data_respuesta['FECHAFIN'] =  pd.to_datetime(data_respuesta['FECHAFIN'])

data['E_mail'] = data['E_mail'].str.strip().str.lower()
data_respuesta['E_MAIL'] = data_respuesta['E_MAIL'].str.strip().str.lower()


# Se unen data y data_respuesta, esto copiara tanto el correo coincidente como la fecha de respuesta de la encuestas
data = data.merge(data_respuesta[['E_MAIL','FECHAFIN']], left_on= 'E_mail', right_on= 'E_MAIL', how= 'left')

# Se recorren coincidencias de email y que se cumpla condicion de fecha de respuesta entre las fechas de envio
data.loc[(data.E_mail == data.E_MAIL) & (data.FECHAFIN >= data.Fecha) & (data.FECHAFIN < data.Fecha_Termino), ['RESPONDIDA']] = 'Sí'
data['RESPONDIDA'] = data['RESPONDIDA'].fillna('No')


#Se guarda la diferencia entre fecha de envio y otra de respuesta, reemplazando el delta entre el rango de fecha (inicio y termino)
data.loc[data['RESPONDIDA'] == 'Sí', ['Duracion']] = (data['FECHAFIN'] - data['Fecha']).map(lambda x: np.nan if pd.isnull(x) else x.days)
data['Duracion'] = data['Duracion'].astype(int)
data['Duracion_i_f'] = data['Fecha_Termino'] - data['Fecha']
data['Duracion_i_f'] = data['Duracion_i_f'].map(lambda x: np.nan if pd.isnull(x) else x.days)
data['Duracion_i_f'] = data['Duracion_i_f'].astype(int)

data = data.drop(data[data['Duracion_i_f'] == 0].index) #Implica presencia de dos envíos, ya que fecha de inicio y termino coinciden
data = data.drop(columns= ['E_MAIL'])
data['FECHAFIN'] = data['FECHAFIN'].fillna('S.F')

data['RESPONDIDA'] = data['RESPONDIDA'].astype(str)

#data['RESPONDIDA'] = data['RESPONDIDA'].replace({'Sí':1,'No':0})
#data['RESPONDIDA'] = data['RESPONDIDA'].astype(int)

data['Fecha'] = data['Fecha'].dt.strftime('%d-%m-%Y').astype(str)
data['Fecha_Termino'] = data['Fecha_Termino'].dt.strftime('%d-%m-%Y').astype(str)
dataset["FECHAFIN"] = dataset["FECHAFIN"].str.split(" ").str.get(0) #En caso de que se guarde formato Año-mes-dia HH:MM:SS, solo se deja fecha
data.to_csv(path + "/DatasetV7.csv", sep= ';', index = False) #Se respalda el conjunto de datos en proceso

Fusión de filas casi duplicadas

In [ ]:
dataset['Duracion'] = dataset['Duracion'].astype(str)
dataset = dataset.groupby(['E_mail', 'Sexo', 'Edad', 'Segmento', 'SubSegmento', 'Segto_Agrup', 'Carterizado', 'Apertura', 'Mes_envio',
                      'Dia_envio', 'Hora_envio', 'Fecha', 'Dia_semana', 'Fecha_Termino']).agg({'FECHAFIN':';'.join, 'Duracion':';'.join, 'RESPONDIDA': ';'.join}).reset_index()

In [ ]:
# Se elimina todos los "No" que provoca inconsistencia en el dataset en caso de haber más de una respuesta encontrada

dataset.loc[(dataset['RESPONDIDA'] == 'No;Sí' ), ['Duracion']] = dataset['Duracion'].str.split(';').str.get(1)
dataset.loc[(dataset['RESPONDIDA'] == 'No;Sí' ), ['FECHAFIN']] = dataset['FECHAFIN'].str.split(';').str.get(1)
dataset.loc[(dataset['RESPONDIDA'] == 'No;Sí' ), ['RESPONDIDA']] = dataset['RESPONDIDA'].str.split(';').str.get(1)

dataset.loc[(dataset['RESPONDIDA'] == 'Sí;No' ), ['Duracion']] = dataset['Duracion'].str.split(';').str.get(0)
dataset.loc[(dataset['RESPONDIDA'] == 'Sí;No' ), ['FECHAFIN']] = dataset['FECHAFIN'].str.split(';').str.get(0)
dataset.loc[(dataset['RESPONDIDA'] == 'Sí;No' ), ['RESPONDIDA']] = dataset['RESPONDIDA'].str.split(';').str.get(0)


dataset.loc[(dataset['RESPONDIDA'] == 'Sí;No;Sí' ), ['Duracion']] = dataset['Duracion'].str.split(';').str.get(0) + ";" + dataset['Duracion'].str.split(';').str.get(2) 
dataset.loc[(dataset['RESPONDIDA'] == 'Sí;No;Sí' ), ['FECHAFIN']] = dataset['FECHAFIN'].str.split(';').str.get(0) + ";" + dataset['FECHAFIN'].str.split(';').str.get(2) 
dataset.loc[(dataset['RESPONDIDA'] == 'Sí;No;Sí' ), ['RESPONDIDA']] = dataset['RESPONDIDA'].str.split(';').str.get(0) + ";" + dataset['RESPONDIDA'].str.split(';').str.get(2) 

dataset.loc[(dataset['RESPONDIDA'] == 'Sí;No;No' ), ['Duracion']] = dataset['Duracion'].str.split(';').str.get(0)
dataset.loc[(dataset['RESPONDIDA'] == 'Sí;No;No' ), ['FECHAFIN']] = dataset['FECHAFIN'].str.split(';').str.get(0)
dataset.loc[(dataset['RESPONDIDA'] == 'Sí;No;No' ), ['RESPONDIDA']] = dataset['RESPONDIDA'].str.split(';').str.get(0)

dataset.loc[(dataset['RESPONDIDA'] == 'No;Sí;No' ), ['Duracion']] = dataset['Duracion'].str.split(';').str.get(1)
dataset.loc[(dataset['RESPONDIDA'] == 'No;Sí;No' ), ['FECHAFIN']] = dataset['FECHAFIN'].str.split(';').str.get(1)
dataset.loc[(dataset['RESPONDIDA'] == 'No;Sí;No' ), ['RESPONDIDA']] = dataset['RESPONDIDA'].str.split(';').str.get(1)

dataset.loc[(dataset['RESPONDIDA'] == 'No;No;Sí' ), ['Duracion']] = dataset['Duracion'].str.split(';').str.get(2)
dataset.loc[(dataset['RESPONDIDA'] == 'No;No;Sí' ), ['FECHAFIN']] = dataset['FECHAFIN'].str.split(';').str.get(2)
dataset.loc[(dataset['RESPONDIDA'] == 'No;No;Sí' ), ['RESPONDIDA']] = dataset['RESPONDIDA'].str.split(';').str.get(2)

dataset.loc[(dataset['RESPONDIDA'] == 'Sí;Sí;No' ), ['Duracion']] = dataset['Duracion'].str.split(';').str.get(0) + ";" + dataset['Duracion'].str.split(';').str.get(1) 
dataset.loc[(dataset['RESPONDIDA'] == 'Sí;Sí;No' ), ['FECHAFIN']] = dataset['FECHAFIN'].str.split(';').str.get(0) + ";" + dataset['FECHAFIN'].str.split(';').str.get(1) 
dataset.loc[(dataset['RESPONDIDA'] == 'Sí;Sí;No' ), ['RESPONDIDA']] = dataset['RESPONDIDA'].str.split(';').str.get(0) + ";" + dataset['RESPONDIDA'].str.split(';').str.get(1) 

dataset.loc[(dataset['RESPONDIDA'] == 'No;Sí;Sí' ), ['Duracion']] = dataset['Duracion'].str.split(';').str.get(1) + ";" + dataset['Duracion'].str.split(';').str.get(2) 
dataset.loc[(dataset['RESPONDIDA'] == 'No;Sí;Sí' ), ['FECHAFIN']] = dataset['FECHAFIN'].str.split(';').str.get(1) + ";" + dataset['FECHAFIN'].str.split(';').str.get(2) 
dataset.loc[(dataset['RESPONDIDA'] == 'No;Sí;Sí' ), ['RESPONDIDA']] = dataset['RESPONDIDA'].str.split(';').str.get(1) + ";" + dataset['RESPONDIDA'].str.split(';').str.get(2) 

dataset.loc[(dataset['RESPONDIDA'] == 'No;No' ), ['Duracion']] = dataset['Duracion'].str.split(';').str.get(0)
dataset.loc[(dataset['RESPONDIDA'] == 'No;No' ), ['FECHAFIN']] = dataset['FECHAFIN'].str.split(';').str.get(0)
dataset.loc[(dataset['RESPONDIDA'] == 'No;No' ), ['RESPONDIDA']] = dataset['RESPONDIDA'].str.split(';').str.get(0)

dataset.loc[(dataset['RESPONDIDA'] == 'No;No;No' ), ['Duracion']] = dataset['Duracion'].str.split(';').str.get(0)
dataset.loc[(dataset['RESPONDIDA'] == 'No;No;No' ), ['FECHAFIN']] = dataset['FECHAFIN'].str.split(';').str.get(0)
dataset.loc[(dataset['RESPONDIDA'] == 'No;No;No' ), ['RESPONDIDA']] = dataset['RESPONDIDA'].str.split(';').str.get(0)

In [ ]:
dataset["RESPONDIDA"].unique()

array(['No', 'Sí', 'Sí;Sí', 'Sí;Sí;Sí'], dtype=object)

Codigo de fuente utilizado: https://www.it-swarm-es.com/es/python/dividir-la-celda-en-varias-filas-en-pandas-dataframe/838453713/

In [ ]:
from itertools import chain

# return list from series of comma-separated strings
def chainer(s):
    return list(chain.from_iterable(s.str.split(';')))

# calculate lengths of splits
lens = dataset['RESPONDIDA'].str.split(';').map(len)

# create new dataframe, repeating or chaining as appropriate
y = pd.DataFrame({'E_mail': np.repeat(dataset['E_mail'], lens),
                  'Sexo': np.repeat(dataset['Sexo'], lens),
                  'Edad': np.repeat(dataset['Edad'], lens),
                  'Segmento': np.repeat(dataset['Segmento'], lens),
                  'SubSegmento': np.repeat(dataset['SubSegmento'], lens),
                  'Segto_Agrup': np.repeat(dataset['Segto_Agrup'], lens),
                  'Carterizado': np.repeat(dataset['Carterizado'], lens),
                  'Apertura': np.repeat(dataset['Apertura'], lens),
                  'Mes_envio': np.repeat(dataset['Mes_envio'], lens),
                  'Dia_envio': np.repeat(dataset['Dia_envio'], lens),
                  'Hora_envio': np.repeat(dataset['Hora_envio'], lens),
                  'Fecha': np.repeat(dataset['Fecha'], lens),
                  'Dia_semana': np.repeat(dataset['Dia_semana'], lens),
                  'Fecha_Termino': np.repeat(dataset['Fecha_Termino'], lens),
                  'FECHAFIN': chainer(dataset['FECHAFIN']),
                  'Duracion': chainer(dataset['Duracion']),
                  'RESPONDIDA': chainer(dataset['RESPONDIDA']),
                  'Duracion_i_f': np.repeat(dataset['Duracion_i_f'], lens})
y = y.replace({'RESPONDIDA' : {'Sí': '1', 'No': '0'}})
y['RESPONDIDA'] = y['RESPONDIDA'].astype(int)

In [ ]:
y = y.reindex(columns=['E_mail','Sexo','Edad','Segmento','SubSegmento','Segto_Agrup', 'Carterizado', 'Apertura', 'Hora_envio', 'Dia_envio', 'Dia_semana', 'Mes_envio', 'Fecha', 'Fecha_Termino', 'FECHAFIN', 'Duracion', 'Duracion_i_f', 'RESPONDIDA'])
y

In [ ]:
y = y.sort_values(['E_mail','Mes_envio', 'Dia_envio']).reset_index(drop = True)
y = y.rename(columns={'FECHAFIN':'Fecha_R'})


#Almacenamiento de los conjuntos de datos

In [ ]:
y = y.sort_values(['E_mail','Fecha']).reset_index(drop = True)
y.to_csv(path + "/Dataset_oficial_VF5.csv", sep= ';', index = False)

In [ ]:
yes = y[y['RESPONDIDA'] == 1]
yes['RESPONDIDA'].unique()

array([1])

In [ ]:
yes['Mes'] = yes['FECHAFIN'].str.split("-").str.get(1).astype(int)
yes.to_csv(path + "/Dataset_oficial_si_respondidas.csv", sep= ';', index = False)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
